In [1]:
import os
import glob
import numpy as np
import xarray as xr
import dask
import time
import itertools

import sys
p = os.path.abspath('/global/homes/q/qnicolas/')
if p not in sys.path:
    sys.path.append(p)

from tools.e5tools import *
xr.set_options(display_style='text') 
from orographicPrecipitation.observationsAndERA5.mountainUtils import BL_plevs
from orographicPrecipitation.precip_model_functions import humidsat,qsat

In [2]:
mountains=         {"vietnam"    :([102,115,10,19.5],"Annamite range (Vietnam)",240,10),
                    "ghats"      :([68,80,7.5,17]   ,"Western Ghats"           ,70 ,6 ),
                    "madagascar" :([40,60,-25,-12]  ,"Madagascar"              ,280,3 ),
                    "myanmar"    :([80,102,7,27]    ,"Myanmar"                 ,60 ,7 ),
                    "newbritain" :([148,155,-12,0]  ,"New Britain"             ,320,7 ),
                    "philippines":([118,130,10,22]  ,"Philippines"             ,225,12),
                    "sumatra"    :([90,110,-6,6]    ,"Bukit Barisan (Sumatra)" ,70 ,11),
                    "malaysia"   :([97,107,1,10]    ,"Malaysia"                ,225,11),
                    "bayofbengal":([82,90,12,18],),
                    "ghatswide"  :([68,85,7.5,17]  ,),
                    "myanmarwide"     :([85,105,12,29]  ,),
                    "newbritainwide"  :([146,155,-12,0]  ,),
                    "smo" : ([240,270,15,35],)
                   }

precip_boxes = {"vietnam"    :[107  ,  19  , 110.5, 14   ,2  ],
                "ghats"      :[ 75  ,   9  ,  72.5, 16   ,2  ],
                "madagascar" :[ 52  , -14.5,  49  , -24.5,2.5],
                "myanmar"    :[ 98  ,  11  ,  90  , 21   ,4  ],
                "newbritain" :[154  , - 6  , 150  , -8   ,2  ],
                "philippines":[123.5,  19  , 127  , 11   ,3  ],
                "sumatra"    :[ 98.5, - 2  ,  96  , 2    ,2.5],
                "malaysia"   :[100.5,  10  , 105  , 4    ,2  ],
                "ghatswide"      :[ 75  ,   9  ,  72.5, 16   ,2  ],
                "myanmarwide"    :[ 98  ,  11  ,  90  , 21   ,4  ],
                "newbritainwide" :[154  , - 6  , 150  , -8   ,2  ],
               }

In [3]:
allyears=range(2001,2021)
months_perregion = {"vietnam"    :[10,11],
                    "ghats"      :[6,7,8],
                    "madagascar" :[3,4],
                    "myanmar"    :[6,7,8],
                    "newbritain" :[6,7,8],
                    "philippines":[11,12],
                    "sumatra"    :[1,11,12],
                    "malaysia"   :[11,12],
                    "ghatswide"   :[6,7,8],
                    "myanmarwide"   :[6,7,8],
                    "newbritainwide":[6,7,8],
                    "smo":[7,8,9],
                   }

#months_perregion = {"vietnam"    :[10],
#                    "ghats"      :[7],
#                    "madagascar" :[3],
#                    "myanmar"    :[7],
#                    "newbritain" :[7],
#                    "philippines":[12],
#                    "sumatra"    :[11],
#                    "malaysia"   :[11],
#                    "bayofbengal":[7],
#                    "ghatswide":[6,8],
#                   }

In [4]:
def tilted_rect(grid,x1,y1,x2,y2,width,reverse=False):
    x = grid.longitude
    y = grid.latitude
    if reverse:
        halfplane_para = (x-x1)*(y2-y1) - (x2-x1)*(y-y1) <=0
    else:
        halfplane_para = (x-x1)*(y2-y1) - (x2-x1)*(y-y1) >=0
    sc_prod = (x-x1)*(x2-x1)+(y-y1)*(y2-y1)
    halfplane_perp_up = sc_prod >= 0
    halfplane_perp_dn = (x-x2)*(x1-x2)+(y-y2)*(y1-y2) >= 0
    distance_across = np.sqrt((x-x1)**2+(y-y1)**2 - sc_prod**2/((x2-x1)**2+(y2-y1)**2))
    return (halfplane_para*halfplane_perp_up*halfplane_perp_dn*(distance_across<width)).transpose('latitude','longitude')
def crossslopeflow(u,v,angle):
    return (u*np.sin(angle*np.pi/180)+v*np.cos(angle*np.pi/180))
    

In [5]:
from dask.distributed import Client
client = Client()

# Loading daily data into zarr groups

## CLIMCAPS

In [7]:
remaining_regions = "madagascar","myanmar","newbritain","philippines","sumatra","malaysia"

In [12]:
years=range(2002,2014)
CLIMCAPS_PATH = "/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/"
for name in ['ghats']:
    print(name)
    box=mountains[name][0]
    months=months_perregion[name]
    for m in months:
        print(m)
        preprocess = lambda ds: ds.swap_dims({'orbit_pass':'obs_time_tai93'}).rename(obs_time_tai93='time',lat='latitude',lon='longitude').spec_hum.isel(time=[1,0])
        ds = xr.open_mfdataset(sorted(glob.glob(CLIMCAPS_PATH+"daily/SNDR.AQUA.AIRS_IM.????{:02}*.L3_CLIMCAPS_QCC.std.v02_38.G.*.nc".format(m))),preprocess=preprocess,combine='nested',concat_dim='time',parallel=True)
        %time ds.reindex(latitude=list(reversed(ds.latitude))).sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_dataset().to_zarr("/global/cscratch1/sd/qnicolas/regionsData/L3_CLIMCAPS_QCC.spec_hum.2002-2015.{:02}.{}.zarr".format(m,name),mode="w")
        
        
        

ghats
7


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 9.04 s, sys: 888 ms, total: 9.93 s
Wall time: 36.8 s


In [99]:
years=range(2002,2014)
CLIMCAPS_PATH = "/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/"

preprocess = lambda ds: ds.air_temp.assign_coords({'time':np.array(ds.obs_time_tai93[0])}).expand_dims('time').rename(lat='latitude',lon='longitude')
ds = xr.open_mfdataset(sorted(glob.glob(CLIMCAPS_PATH+"monthly/SNDR.AQUA.AIRS_IM.*.nc")),preprocess=preprocess,combine='nested',concat_dim='time',parallel=True)
%time ds.reindex(latitude=list(reversed(ds.latitude))).to_dataset().to_zarr(CLIMCAPS_PATH+"monthly/L3_CLIMCAPS_QCC.air_temp.monthly.zarr",mode="w")
        
        
        

/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 3.37 s, sys: 301 ms, total: 3.67 s
Wall time: 14.8 s


In [6]:
years=range(2002,2016)
CLIMCAPS_PATH = "/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/"
for name in ['myanmarwide','newbritainwide']:
    print(name)
    box=mountains[name][0]
    months=months_perregion[name]
    for m in months:
        print(m)
        preprocess = lambda ds: ds.swap_dims({'orbit_pass':'obs_time_tai93'}).rename(obs_time_tai93='time',lat='latitude',lon='longitude').air_temp.isel(time=[1,0])
        ds = xr.open_mfdataset(sorted(glob.glob(CLIMCAPS_PATH+"daily/SNDR.AQUA.AIRS_IM.????{:02}*.L3_CLIMCAPS_QCC.std.v02_38.G.*.nc".format(m))),preprocess=preprocess,combine='nested',concat_dim='time',parallel=True)
        %time ds.reindex(latitude=list(reversed(ds.latitude))).sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_dataset().to_zarr("/global/cscratch1/sd/qnicolas/regionsData/L3_CLIMCAPS_QCC.air_temp.2002-2015.{:02}.{}.zarr".format(m,name),mode="w")
        
        
        

myanmarwide
6


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 7.65 s, sys: 988 ms, total: 8.64 s
Wall time: 34.4 s
7


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 7.1 s, sys: 914 ms, total: 8.01 s
Wall time: 34.6 s
8


...

CPU times: user 7.11 s, sys: 833 ms, total: 7.94 s
Wall time: 37.7 s
7


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 8.72 s, sys: 859 ms, total: 9.57 s
Wall time: 43.9 s
8


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 8.11 s, sys: 902 ms, total: 9.01 s
Wall time: 41.8 s


## ERA5 plevs

In [6]:
def createzarr_era5_pl(name,m,varcode):
    box=mountains[name][0]
    years=allyears
    for y in years:
        filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name)
        if os.path.isdir(filepath):
            print("Already exists! - %s"%filepath)
            continue
        print("Doesn't exist! - %s"%filepath)
        var = xr.open_mfdataset(ERA5D_PATH+"e5.oper.an.pl/{}{:02}/e5.oper.an.pl.{}.*.nc".format(y,m,varcode))
        var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr(filepath,mode="w")
        

In [ ]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","philippines","malaysia"):
    months=months_perregion[name]
    for varcode in ["128_131_u","128_132_v"]:
        for m in months:
            %time createzarr_era5_pl(name,m,varcode)

Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200106.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200206.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200306.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200406.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200506.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200606.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200706.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll025sc.200806.ghatswide.zarr
Already exists! - /global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_131_u.ll02

## ERA5 sfc - TRMM - GPM

In [7]:
def createzarr_era5_sfc(name,m,varcode):
    box=mountains[name][0]
    years=allyears
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isfile(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.sfc/*/e5.oper.an.sfc.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr(filepath,mode="w")
        
def createzarr_trmm(name,m):
    box=mountains[name][0]
    years=range(2010,2014)

    var = xr.open_mfdataset([glob.glob(CMIP6_FOLDER+"obs4mip/NASA-GSFC/TRMM/observations/atmos/pr/3hr/NASA-GSFC/TRMM/*/*_{}{:02}*.nc".format(y,m))[0] for y in years])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr("/global/cscratch1/sd/qnicolas/regionsData/trmm.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name),mode="w")

        

In [10]:
def flatten(t):
    return [item for sublist in t for item in sublist]
def createzarr_gpm(name,m):
    box=mountains[name][0]
    years=allyears
    
    var = xr.open_mfdataset([glob.glob("/global/cscratch1/sd/pnicknis/gpm/daily/3B-DAY.MS.MRG.3IMERG.{}{:02}*-S000000-E235959.V06.nc4".format(y,m)) for y in years])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    var = var.assign_coords({'longitude': var.longitude%360})
    var = var.sortby('longitude')
    var['time'] = var.indexes['time'].to_datetimeindex()
    var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr("/global/cscratch1/sd/qnicolas/regionsData/gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name),mode="w")


In [6]:
def createzarr_era5_vinteg(name,m,varcode):
    box=mountains[name][0]
    years=allyears
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isfile(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.vinteg/*/e5.oper.an.vinteg.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr(filepath,mode="w")
        

## To be run

In [11]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
        print(name)
        months=months_perregion[name]
        for varcode in ['228_246_100u','228_247_100v','128_137_tcwv']:
            print(varcode)
            for m in months:
                print(m)
                %time createzarr_era5_sfc(name,m,varcode)

ghatswide
228_246_100u
6
CPU times: user 2.67 s, sys: 1.11 s, total: 3.78 s
Wall time: 20.8 s
7
CPU times: user 2.06 s, sys: 1.18 s, total: 3.25 s
Wall time: 14 s
8
CPU times: user 1.87 s, sys: 994 ms, total: 2.86 s
Wall time: 12.8 s
228_247_100v
6
CPU times: user 2.05 s, sys: 1.02 s, total: 3.06 s
Wall time: 13.4 s
7
CPU times: user 2.16 s, sys: 906 ms, total: 3.06 s
Wall time: 14.8 s
8
CPU times: user 2.03 s, sys: 1.04 s, total: 3.07 s
Wall time: 13.7 s
128_137_tcwv
6
CPU times: user 2.04 s, sys: 1.07 s, total: 3.1 s
Wall time: 11.6 s
7
CPU times: user 1.85 s, sys: 939 ms, total: 2.79 s
Wall time: 11.1 s
8
CPU times: user 2.02 s, sys: 916 ms, total: 2.93 s
Wall time: 10.9 s
myanmarwide
228_246_100u
6
CPU times: user 1.94 s, sys: 1.04 s, total: 2.98 s
Wall time: 12.8 s
7
CPU times: user 2.04 s, sys: 879 ms, total: 2.92 s
Wall time: 13.8 s
8
CPU times: user 1.96 s, sys: 871 ms, total: 2.83 s
Wall time: 12.8 s
228_247_100v
6
CPU times: user 1.94 s, sys: 1.06 s, total: 3 s
Wall time: 13.

In [7]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
        print(name)
        months=months_perregion[name]
        for varcode in ['162_071_viwve','162_072_viwvn']:
            print(varcode)
            for m in months:
                print(m)
                %time createzarr_era5_vinteg(name,m,varcode)

ghatswide
162_071_viwve
6
CPU times: user 2.27 s, sys: 1.23 s, total: 3.5 s
Wall time: 13.9 s
8
CPU times: user 2.36 s, sys: 1.41 s, total: 3.76 s
Wall time: 13 s
162_072_viwvn
6
CPU times: user 2.29 s, sys: 1.2 s, total: 3.5 s
Wall time: 13.3 s
7
CPU times: user 2.37 s, sys: 1.2 s, total: 3.57 s
Wall time: 14.8 s
8
CPU times: user 1.95 s, sys: 1.18 s, total: 3.14 s
Wall time: 10.3 s
newbritainwide
162_071_viwve
6
CPU times: user 2.08 s, sys: 1.42 s, total: 3.51 s
Wall time: 7.45 s
7
CPU times: user 1.87 s, sys: 1.28 s, total: 3.15 s
Wall time: 8.16 s
8
CPU times: user 1.78 s, sys: 1.34 s, total: 3.12 s
Wall time: 9.15 s
162_072_viwvn
6
CPU times: user 1.73 s, sys: 1.27 s, total: 3 s
Wall time: 9.47 s
7
CPU times: user 1.79 s, sys: 1.22 s, total: 3 s
Wall time: 8.88 s
8
CPU times: user 1.86 s, sys: 1.24 s, total: 3.1 s
Wall time: 9.8 s
vietnam
162_071_viwve
10
CPU times: user 1.96 s, sys: 1.48 s, total: 3.44 s
Wall time: 12.1 s
11
CPU times: user 2.06 s, sys: 1.29 s, total: 3.35 s
Wall

In [14]:
def extendzarr_gpm(name,m):
    box=mountains[name][0]
    years_sub=range(2016,2021)
    
    ori_filename = "/global/cscratch1/sd/qnicolas/regionsData/gpm_imerg_v06.2001-2015.{:02}.{}.zarr".format(m,name)
    new_filename = "/global/cscratch1/sd/qnicolas/regionsData/gpm_imerg_v06.2001-2020.{:02}.{}.zarr".format(m,name)
    

    var = xr.open_mfdataset([glob.glob("/global/cscratch1/sd/pnicknis/gpm/daily/3B-DAY.MS.MRG.3IMERG.{}{:02}*-S000000-E235959.V06.nc4".format(y,m)) for y in years_sub])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    var = var.assign_coords({'longitude': var.longitude%360})
    var = var.sortby('longitude')
    var['time'] = var.indexes['time'].to_datetimeindex()
    var = var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2]))
    
    xr.concat((xr.open_zarr(ori_filename),var),dim='time').to_zarr(new_filename,mode="w")

with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for couplet in [('ghatswide',8),
                    ('myanmarwide',6),('myanmarwide',7),('myanmarwide',8),
                    ('newbritainwide',6),('newbritainwide',7),('newbritainwide',8),
                   ]:
        print(couplet)
        %time extendzarr_gpm(*couplet)

('ghatswide', 8)


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  


CPU times: user 28.4 s, sys: 5.07 s, total: 33.4 s
Wall time: 1min 11s
('myanmarwide', 6)


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  


CPU times: user 26.3 s, sys: 4.43 s, total: 30.7 s
Wall time: 1min 1s
('myanmarwide', 7)


...

CPU times: user 27.1 s, sys: 1.48 s, total: 28.6 s
Wall time: 44.6 s
('newbritainwide', 7)


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  


CPU times: user 26.5 s, sys: 1.71 s, total: 28.2 s
Wall time: 50.2 s
('newbritainwide', 8)


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  


CPU times: user 27 s, sys: 1.65 s, total: 28.6 s
Wall time: 58.7 s


In [11]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for varcode in ['228_246_100u','228_247_100v']:
        print(varcode)
        %time createzarr_era5_sfc('smo',8,varcode)
    print('gpm precip')
    %time createzarr_gpm('smo',8)

228_246_100u
CPU times: user 8.95 s, sys: 2.09 s, total: 11 s
Wall time: 2min 11s
228_247_100v
CPU times: user 6.39 s, sys: 1.86 s, total: 8.25 s
Wall time: 1min 26s
gpm precip


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  if sys.path[0] == '':


CPU times: user 2min 11s, sys: 24.9 s, total: 2min 36s
Wall time: 7min 20s


# Time and space mean

## Daily

In [17]:
def resample_daily(ds):
    """performs the same job as resample(time="1D") but keeps missing days out"""
    ds.coords['time'] = ds.time.dt.floor('1D')
    return ds.groupby('time').mean()

In [7]:
def compute_spatialmean_pl(name,varcode):
    years = allyears
    months= months_perregion[name]
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        ds = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name))
        varname = list(ds.data_vars)[0]
        ds=ds[varname]
        mask_era5= tilted_rect(ds,*precip_boxes[name],reverse=False)
        spatialmeans.append(spatial_mean(resample_daily(ds),box=None,mask=mask_era5))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}-{}.{}.{}.abovemean.nc".format(varcode,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 

In [8]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
    %time compute_spatialmean_pl(name,'128_130_t')

CPU times: user 40.8 s, sys: 2.72 s, total: 43.5 s
Wall time: 1min 6s
CPU times: user 44.9 s, sys: 2.99 s, total: 47.9 s
Wall time: 1min 15s
CPU times: user 40.2 s, sys: 2.38 s, total: 42.5 s
Wall time: 1min 3s
CPU times: user 26.1 s, sys: 1.53 s, total: 27.6 s
Wall time: 41.1 s
CPU times: user 27.7 s, sys: 1.6 s, total: 29.3 s
Wall time: 44.2 s
CPU times: user 26.1 s, sys: 1.44 s, total: 27.5 s
Wall time: 41.2 s
CPU times: user 43.1 s, sys: 2.57 s, total: 45.7 s
Wall time: 1min 8s
CPU times: user 26.6 s, sys: 1.42 s, total: 28.1 s
Wall time: 40.8 s


In [7]:
def compute_upstream_spatialmean_sfc(name,varcode):
    years=allyears
    months=months_perregion[name]
    var_months = [xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*precip_boxes[name],reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.{}.ll025sc.{}-{}.{}.{}.upstreammean.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def compute_spatialmean_gpm(name):
    years=allyears
    months=months_perregion[name]
    var_months = [xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)) for m in months]
    mask_ghats_gpm= tilted_rect(var_months[0],*precip_boxes[name],reverse=False)
    var_months_sm  = [spatial_mean(var.precipitationCal,box=None,mask=mask_ghats_gpm) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/gpm_imerg_v06.{}-{}.{}.{}.abovemean.nc".format(years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 
    

In [8]:
def compute_upstream_spatialmean_vinteg(name,varcode):
    years=allyears
    months=months_perregion[name]
    var_months = [xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*precip_boxes[name],reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.vinteg.{}.ll025sc.{}-{}.{}.{}.upstreammean.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 


In [ ]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
    print(name)
    for varcode in ['162_071_viwve','162_072_viwvn']:
        %time compute_upstream_spatialmean_vinteg(name,varcode)

ghatswide
CPU times: user 30.7 s, sys: 872 ms, total: 31.6 s
Wall time: 35 s
newbritainwide
CPU times: user 29.8 s, sys: 815 ms, total: 30.6 s
Wall time: 34.4 s
CPU times: user 30 s, sys: 882 ms, total: 30.9 s
Wall time: 34.8 s
vietnam
CPU times: user 19.1 s, sys: 609 ms, total: 19.7 s
Wall time: 22.5 s
CPU times: user 19.6 s, sys: 593 ms, total: 20.2 s
Wall time: 23.4 s
CPU times: user 19.3 s, sys: 541 ms, total: 19.8 s
Wall time: 22 s
sumatra
CPU times: user 29.8 s, sys: 1.02 s, total: 30.8 s
Wall time: 34.2 s
CPU times: user 30.2 s, sys: 1.01 s, total: 31.2 s
Wall time: 34.6 s
malaysia
CPU times: user 19.5 s, sys: 675 ms, total: 20.1 s
Wall time: 22.1 s


In [42]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
    print(name)
    %time compute_spatialmean_gpm(name)

myanmarwide
CPU times: user 28 s, sys: 1.44 s, total: 29.5 s
Wall time: 46.3 s
newbritainwide
CPU times: user 26.9 s, sys: 1.27 s, total: 28.2 s
Wall time: 36 s
vietnam
CPU times: user 19.5 s, sys: 848 ms, total: 20.3 s
Wall time: 33.2 s
madagascar
CPU times: user 18.3 s, sys: 864 ms, total: 19.1 s
Wall time: 24.3 s
philippines
CPU times: user 18.7 s, sys: 918 ms, total: 19.7 s
Wall time: 28.6 s
sumatra
CPU times: user 28.8 s, sys: 1.22 s, total: 30 s
Wall time: 37.7 s
malaysia
CPU times: user 19.3 s, sys: 856 ms, total: 20.2 s
Wall time: 30.8 s


In [7]:
def thetae_perso(t,q,es,qs,pname='level'):
    """t in K, t[pname] and es in hPa, q and qs in kg/kg. returns thetae in K"""
    r  = q/(1-q)
    rh = q/qs
    # Magnus formula for dew point
    b=18.678;c=257.14
    gamma = np.log(rh)+b*(t-275.15)/(c+(t-273.15))
    tdew = 273.15+c*gamma/(b-gamma)
    tL = 1/(1/(tdew-56)+np.log(t/tdew)/800)+56
    e = rh*es
    
    kappad=0.2854
    thetaL = t*(1e3/(t[pname]-e))**kappad*(t/tL)**(0.28*r)
    thetae = thetaL*np.exp((3036/tL-1.78)*r*(1+0.448*r))
    return thetae

def thetaestar_perso(t,es,qs,pname='level'):
    """t in K, t[pname] and es in hPa, qs in kg/kg. returns thetae^* in K"""
    rs=qs/(1-qs)
    kappad=0.2854
    thetaL = t*(1e3/(t[pname]-es))**kappad
    thetae = thetaL*np.exp((3036/t-1.78)*rs*(1+0.448*rs))
    return thetae

############################################################
#def thetae_perso_approx(t,q,pname='level'):
#    r=q/(1-q)
#    thetae = (t+2.5e6/1004.*r)*(1e3/t[pname])**(287/1004.)
#    return thetae
#def thetaestar_perso_approx(t,pname='level'):
#    qs = qsat(t,t[pname])
#    rs=qs/(1-qs)
#    thetaestar = (t+2.5e6/1004.*rs)*(1e3/t[pname])**(287/1004.)
#    return thetaestar


In [8]:
from orographicPrecipitation.precip_model_functions import humidsat
def BL_plevs_vectorized(t,q,lft_top_pressure,pname='level',fixed_thetaeb=False):
    kappaL=3
    g=9.81
    wB = 0.52
    wL = 1-wB
    thetae0 = 340
    t = t.sel({pname:slice(lft_top_pressure,1100.)})
    q = q.sel({pname:slice(lft_top_pressure,1100.)})
    
    es,qs,_=humidsat(t,t[pname])
    thetae =     thetae_perso(t,q,es,qs,pname=pname)
    thetaestar = thetaestar_perso(t,es,qs,pname=pname)  
        
    if fixed_thetaeb:
        thetaeB = 350
    else:
        thetaeB = thetae    .sel({pname:slice(900,1100)            }).mean(pname)
    thetaeL     = thetae    .sel({pname:slice(lft_top_pressure,899)}).mean(pname)
    thetaeLstar = thetaestar.sel({pname:slice(lft_top_pressure,899)}).mean(pname)
    
    capeL   = (thetaeB/thetaeLstar - 1)*thetae0
    subsatL = (1 - thetaeL/thetaeLstar)*thetae0
    BL = g/kappaL/thetae0*(wB*capeL-wL*subsatL)
    return BL

def thetaeb_vectorized(t,q,pname='level'):
    t = t.sel({pname:slice(850.,1100.)}) #900.
    q = q.sel({pname:slice(850.,1100.)}) #900.
    es,qs,_=humidsat(t,t[pname])
    thetae =     thetae_perso(t,q,es,qs,pname=pname)
    return thetae.mean(pname)

def thetaeL_vectorized(t,q,lft_top_pressure,pname='level'):
    t = t.sel({pname:slice(lft_top_pressure,849.)})
    q = q.sel({pname:slice(lft_top_pressure,849.)})
    es,qs,_=humidsat(t,t[pname])
    thetae =     thetae_perso(t,q,es,qs,pname=pname)
    return thetae.mean(pname)

def thetaeLstar_vectorized(t,lft_top_pressure,pname='level'):
    t = t.sel({pname:slice(lft_top_pressure,849.)})
    es,qs,_=humidsat(t,t[pname])
    thetaestar = thetaestar_perso(t,es,qs,pname=pname)  
    return thetaestar.mean(pname)

In [14]:
def compute_spatialmean_Bl(name):
    years = allyears
    months= months_perregion[name]
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        mask_era5= tilted_rect(q,*precip_boxes[name],reverse=False)
        Bl = BL_plevs_vectorized(t,q,700.,pname='level',fixed_thetaeb=False)
        spatialmeans.append(spatial_mean(resample_daily(Bl),box=None,mask=mask_era5))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.Bl.{}-{}.{}.{}.abovemean.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 

In [15]:
def compute_thetaeb(name):
    years = allyears
    months= months_perregion[name]
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        thetaeb = thetaeb_vectorized(t,q,pname='level')
        spatialmeans.append(resample_daily(thetaeb))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.thetaebdeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [16]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
    print(name)
    %time compute_thetaeb(name)

ghatswide
2001 6
2001 7
2001 8
2002 6
2002 7
2002 8
2003 6
2003 7
2003 8
2004 6
2004 7
2004 8
2005 6
2005 7
2005 8
2006 6
2006 7
2006 8
2007 6
2007 7
2007 8
2008 6
2008 7
2008 8
2009 6
2009 7
2009 8
2010 6
2010 7
2010 8
2011 6
2011 7
2011 8
2012 6
2012 7
2012 8
2013 6
2013 7
2013 8
2014 6
2014 7
2014 8
2015 6
2015 7
2015 8
2016 6
2016 7
2016 8
2017 6
2017 7
2017 8
2018 6
2018 7
2018 8
2019 6
2019 7
2019 8
2020 6
2020 7
2020 8
CPU times: user 1min 16s, sys: 4.72 s, total: 1min 21s
Wall time: 2min 12s
myanmarwide
2001 6
2001 7
2001 8
2002 6
2002 7
2002 8
2003 6
2003 7
2003 8
2004 6
2004 7
2004 8
2005 6
2005 7
2005 8
2006 6
2006 7
2006 8
2007 6
2007 7
2007 8
2008 6
2008 7
2008 8
2009 6
2009 7
2009 8
2010 6
2010 7
2010 8
2011 6
2011 7
2011 8
2012 6
2012 7
2012 8
2013 6
2013 7
2013 8
2014 6
2014 7
2014 8
2015 6
2015 7
2015 8
2016 6
2016 7
2016 8
2017 6
2017 7
2017 8
2018 6
2018 7
2018 8
2019 6
2019 7
2019 8
2020 6
2020 7
2020 8
CPU times: user 1min 32s, sys: 6.7 s, total: 1min 39s
Wall time

In [9]:
def compute_thetaeL(name):
    years = allyears
    months= months_perregion[name]
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        thetaeL = thetaeL_vectorized(t,q,500.,pname='level')
        spatialmeans.append(resample_daily(thetaeL))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.thetaeLdeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [ ]:
for name in ('ghatswide',):
    print(name)
    %time compute_thetaeL(name)

In [19]:
def compute_thetaeLstar(name):
    years = allyears
    months= months_perregion[name]
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        t = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        thetaeL = thetaeLstar_vectorized(t,500.,pname='level')
        spatialmeans.append(resample_daily(thetaeL))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.thetaeLstardeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [ ]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
    print(name)
    %time compute_thetaeLstar(name)

In [7]:
def compute_ebeLeLstar(name):
    years = allyears
    months= months_perregion[name]
    ebs=[]
    eLs=[]
    eLstars=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        eb = 2.5e6/1004*q.sel(level=slice(900,1100))+t.sel(level=slice(900,1100))
        tL = t.sel(level=slice(700.,899.))
        qL = q.sel(level=slice(700.,899.))
        _,qs,_=humidsat(tL,tL.level)
        eL = 2.5e6/1004*qL+tL
        eLstar = 2.5e6/1004*qs+tL
        
        ebs.append(resample_daily(eb))
        eLs.append(resample_daily(eL))
        eLstars.append(resample_daily(eLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(ebs,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.eb.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLs,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.eL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLstars,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.eLstar.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [13]:
def compute_tLqL(name):
    years = allyears
    months= months_perregion[name]
    tLs=[]
    qLs=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        tL = t.sel(level=slice(700.,899.)).mean('level')
        qL = q.sel(level=slice(700.,899.)).mean('level')
        tLs.append(resample_daily(tL))
        qLs.append(resample_daily(qL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(tLs,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.tL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(qLs,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.qL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [ ]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","madagascar","philippines","sumatra","malaysia"):
    print(name)
    %time compute_tLqL(name)

In [15]:
def compute_uLvL(name):
    years = allyears
    months= months_perregion[name]
    uLs=[]
    vLs=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        u = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_131_u',y,m,name)).U
        v = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_132_v',y,m,name)).V
        uL = u.sel(level=slice(700.,899.)).mean('level')
        vL = v.sel(level=slice(700.,899.)).mean('level')
        uLs.append(resample_daily(uL))
        vLs.append(resample_daily(vL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(uLs,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.uL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(vLs,dim='time')
    filepath = "/global/cscratch1/sd/qnicolas/regionsData/e5.diagnostic.vL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

# U,V

In [21]:
for name in ('ghatswide','myanmarwide','newbritainwide',"vietnam","philippines","malaysia"):
    years = allyears
    months= months_perregion[name]
    uLs=[]
    vLs=[]
    for y,m in itertools.product(years,months):
        pathu = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_131_u',y,m,name)
        pathv = "/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_132_v',y,m,name)
        if not os.path.isdir(pathu):
            print("u, {}, {}{:02}".format(name,y,m))
        if not os.path.isdir(pathu):
            print("v, {}, {}{:02}".format(name,y,m))

u, ghatswide, 201606
v, ghatswide, 201606
u, ghatswide, 201607
v, ghatswide, 201607
u, ghatswide, 201608
v, ghatswide, 201608
u, ghatswide, 201706
v, ghatswide, 201706
u, ghatswide, 201707
v, ghatswide, 201707
u, ghatswide, 201708
v, ghatswide, 201708
u, ghatswide, 201806
v, ghatswide, 201806
u, ghatswide, 201807
v, ghatswide, 201807
u, ghatswide, 201808
v, ghatswide, 201808
u, ghatswide, 201906
v, ghatswide, 201906
u, ghatswide, 201907
v, ghatswide, 201907
u, ghatswide, 201908
v, ghatswide, 201908
u, ghatswide, 202006
v, ghatswide, 202006
u, ghatswide, 202007
v, ghatswide, 202007
u, ghatswide, 202008
v, ghatswide, 202008
u, myanmarwide, 201606
v, myanmarwide, 201606
u, myanmarwide, 201607
v, myanmarwide, 201607
u, myanmarwide, 201608
v, myanmarwide, 201608
u, myanmarwide, 201706
v, myanmarwide, 201706
u, myanmarwide, 201707
v, myanmarwide, 201707
u, myanmarwide, 201708
v, myanmarwide, 201708
u, myanmarwide, 201806
v, myanmarwide, 201806
u, myanmarwide, 201807
v, myanmarwide, 201807
u,